In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 40.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5

In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-25 16:29:36.905647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750868977.375220      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750868977.511686      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from unsloth import FastLanguageModel
from peft import LoraConfig

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "microsoft/phi-2",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True, #quantization
)

==((====))==  Unsloth 2025.6.5: Fast Phi patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

microsoft/phi-2 does not have a padding token! Will use pad_token = <|endoftext|>.


In [4]:
dataset = load_dataset("b-mc2/sql-create-context", split="train[:50000]")
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [5]:
def format_input(input):
    prompt = f"Context: {input['context']}\n\nQuestion: {input['question']}\n\nAnswer:"
    response = input['answer']
    prmpt = (
                "### Instruction:\nGenerate SQL query based on the following context and question.\n\n"
                f"### Context:\n{input['context']}\n\n"
                f"### Question:\n{input['question']}\n\n"
                "### Response:\n"
            )
    return {"text": prompt + " " + response + tokenizer.eos_token}


## on formate la dataset

In [6]:
formatted_dataset = dataset.map(format_input, remove_columns=["context", "question", "answer"])
formatted_dataset

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 5000
    })
})

In [7]:
formatted_dataset['train'][0]

{'text': 'Context: CREATE TABLE table_name_52 (losses INTEGER, against VARCHAR, wins VARCHAR)\n\nQuestion: What was the greatest number of Losses for the team that had 1427 Against and more than 5 Wins?\n\nAnswer: SELECT MAX(losses) FROM table_name_52 WHERE against > 1427 AND wins > 5<|endoftext|>'}

## configurer lora

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    #use_gradient_checkpointing=True,
    random_state=3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


## tester les performence du model avant finetuning:

In [9]:
def generate_sql(context, question):
    prompt = f"""### Instruction:
Generate SQL query based on the following context and question.

### Context:
{context}

### Question:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with some examples
test_cases = [
    {
        "context": "CREATE TABLE Sportsinfo (sportname VARCHAR, onscholarship VARCHAR)",
        "question": "Which sport has most number of students on scholarship?"
    },
    {
        "context": "CREATE TABLE Students (id INT, name VARCHAR, grade INT)",
        "question": "Find all students with grade greater than 80"
    }
]

for test in test_cases:
    result = generate_sql(test["context"], test["question"])
    print("\nContext:", test["context"])
    print("Question:", test["question"])
    print("Generated SQL:", result)


Context: CREATE TABLE Sportsinfo (sportname VARCHAR, onscholarship VARCHAR)
Question: Which sport has most number of students on scholarship?
Generated SQL: ### Instruction:
Generate SQL query based on the following context and question.

### Context:
CREATE TABLE Sportsinfo (sportname VARCHAR, onscholarship VARCHAR)

### Question:
Which sport has most number of students on scholarship?

### Response:
```sql
SELECT sportname, COUNT(*) AS scholarship FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (SELECT COUNT(*) FROM Sportsinfo GROUP BY sportname HAVING COUNT(scholarship) = (S

### on remarque que le model avant d'etre entrainer il fait des fautes au niveau de generation de code sql donc il doit etre entrainer sur la dataset "sql-create-context" pour lui donner de la valeur ajouté 

### setup de wandb:

In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandbapi")

In [11]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedach (mohammedach-adobe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
wandb.init(project="phi2-sql-finetuning-v3", name="phi2_sql_run_v3")

### phase de configuration de sfttrainer et tokenisation du dataset

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
import torch
from unsloth import is_bfloat16_supported
import logging
logging.basicConfig(level=logging.INFO)



training_args = TrainingArguments(
    per_device_train_batch_size=6,                
    gradient_accumulation_steps=4,                
    num_train_epochs=2,                           
    warmup_ratio=0.05,                            
    learning_rate=2e-5,                           
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=25,                             
    optim="adamw_8bit",                           
    weight_decay=0.01,
    lr_scheduler_type="cosine",                   
    seed=3407,
    output_dir="outputs",
    logging_dir="outputs/logs",

    # Unsloth compatibility with hardware
    use_mps_device=False,
    use_cpu=False,
    no_cuda=False,
    gradient_checkpointing=True,                  
    torch_compile=False,
    use_legacy_prediction_loop=False,             

    # Evaluation & saving
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,                               # Every 1/4 epoch
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=2,

    # Logging
    report_to="wandb",
    run_name="phi2-sql-finetuning-v3"
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset['train'],
    eval_dataset=formatted_dataset["test"],
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=training_args,
    
)



Unsloth: Tokenizing ["text"]:   0%|          | 0/45000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [14]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 45,000 | Num Epochs = 2 | Total steps = 1,874
O^O/ \_/ \    Batch size per device = 12 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (12 x 4 x 1) = 48
 "-____-"     Trainable parameters = 10,485,760/1,531,878,400 (0.68% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,1.705700,1.419090
200,1.198500,1.136664
300,1.116500,1.080570
400,1.078000,1.046776
500,1.047700,1.020715
600,1.044100,1.002901
700,1.029200,0.991618
800,1.011800,0.982363
900,1.005500,0.972243
1000,0.986000,0.966470


Unsloth: Not an error, but PhiForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=1874, training_loss=1.0882491637955607, metrics={'train_runtime': 17985.1787, 'train_samples_per_second': 5.004, 'train_steps_per_second': 0.104, 'total_flos': 1.669436144326656e+17, 'train_loss': 1.0882491637955607})

In [15]:
trainer.save_model("./final_model_v3")
tokenizer.save_pretrained("./final_model_v3")

('./final_model_v3/tokenizer_config.json',
 './final_model_v3/special_tokens_map.json',
 './final_model_v3/vocab.json',
 './final_model_v3/merges.txt',
 './final_model_v3/added_tokens.json',
 './final_model_v3/tokenizer.json')

### publier le model dans huggingface

In [16]:
secret_value_1 = user_secrets.get_secret("huggingface")

In [17]:
from huggingface_hub import login
login(token=secret_value_1)

In [18]:
model.push_to_hub(
    "medachbab/phi2-sql-v3",
    private=True
)

tokenizer.push_to_hub(
    "medachbab/phi2-sql-v3",
    private=True
)

README.md:   0%|          | 0.00/548 [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Saved model to https://huggingface.co/medachbab/phi2-sql-v3


In [20]:

model.save_pretrained("saved_model_v3")


tokenizer.save_pretrained("saved_model_v3")

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.json',
 'saved_model/merges.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')